<h1 style="text-align: center;">Text Classification</h1>

Text Classification is one of the basic and most important task of Natural Language Processing. In this notebook, I am focussing on Sentiment Analysis task using CNN model.

**Reference**

[Convolutional Neural Networks for Sentence Classification](https://arxiv.org/pdf/1408.5882)

In [1]:
import os
import sys
import time
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np

from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe

### 1 Data Preparation

In [2]:
def load_dataset(test_sen=None):

    """
    tokenizer : Breaks sentences into a list of words. If sequential=False, no tokenization is applied
    Field : A class that stores information about the way of preprocessing
    fix_length : An important property of TorchText is that we can let the input to be variable length, and TorchText will
                 dynamically pad each sequence to the longest sequence in that "batch". But here we are using fi_length which
                 will pad each sequence to have a fix length of 200.
                 
    build_vocab : It will first make a vocabulary or dictionary mapping all the unique words present in the train_data to an
                  idx and then after it will use GloVe word embedding to map the index to the corresponding word embedding.
                  
    vocab.vectors : This returns a torch tensor of shape (vocab_size x embedding_dim) containing the pre-trained word embeddings.
    BucketIterator : Defines an iterator that batches examples of similar lengths together to minimize the amount of padding needed.
    
    """
    # Define tokenizer function to split text into words
    tokenize = lambda x: x.split()
    # Create text field processor with configurations:
    # - sequential: Process as sequence data
    # - tokenize: Use the defined tokenizer
    # - lower: Convert to lowercase
    # - include_lengths: Keep sequence length info
    # - batch_first: Put batch dimension first
    # - fix_length: Pad all sequences to 200 words
    TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True, fix_length=200)
    # Create label field processor for sentiment labels (0/1)
    LABEL = data.LabelField(dtype=torch.long)
    # Load IMDB dataset and split into train/test sets
    train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
    # Build vocabulary using training data and load GloVe embeddings
    TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))
    LABEL.build_vocab(train_data)


    word_embeddings = TEXT.vocab.vectors
    print ("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
    print ("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
    print ("Label Length: " + str(len(LABEL.vocab)))
    
    # Print number of samples for each label
    print("Number of samples in each class:")
    for label, count in LABEL.vocab.freqs.items():
        print(f"Class {label}: {count} samples")

    train_data, valid_data = train_data.split() # Further splitting of training_data to create new training_data & validation_data
    train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=32, sort_key=lambda x: len(x.text), repeat=False, shuffle=True)

    '''Alternatively we can also use the default configurations'''
    # train_iter, test_iter = datasets.IMDB.iters(batch_size=32)

    vocab_size = len(TEXT.vocab)

    return TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter

# Load the dataset
# We use pre-trained word embeddings as weights for our CNN
TEXT, vocab_size, word_embeddings, train_iter, valid_iter, test_iter = load_dataset()

Length of Text Vocabulary: 251639
Vector size of Text Vocabulary:  torch.Size([251639, 300])
Label Length: 2
Number of samples in each class:
Class pos: 12500 samples
Class neg: 12500 samples


### **2 CNN Model**

In [3]:
class CNN(nn.Module):
	def __init__(self, batch_size, output_size, in_channels, out_channels, kernel_heights, stride, padding, keep_probab, vocab_size, embedding_length, weights):
		super(CNN, self).__init__()
		
		"""
		Arguments
		---------
		batch_size : Size of each batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		in_channels : Number of input channels. Here it is 1 as the input data has dimension = (batch_size, num_seq, embedding_length)
		out_channels : Number of output channels after convolution operation performed on the input matrix
		kernel_heights : A list consisting of 3 different kernel_heights. Convolution will be performed 3 times and finally results from each kernel_height will be concatenated.
		keep_probab : Probability of retaining an activation node during dropout operation
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embedding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table
		--------
		
		"""
		self.batch_size = batch_size
		self.output_size = output_size
		self.in_channels = in_channels
		self.out_channels = out_channels
		self.kernel_heights = kernel_heights
		self.stride = stride
		self.padding = padding
		self.vocab_size = vocab_size
		self.embedding_length = embedding_length
		
		self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
		self.word_embeddings.weight = nn.Parameter(weights, requires_grad=False)
		self.conv1 = nn.Conv2d(in_channels, out_channels, (kernel_heights[0], embedding_length), stride, padding)
		self.conv2 = nn.Conv2d(in_channels, out_channels, (kernel_heights[1], embedding_length), stride, padding)
		self.conv3 = nn.Conv2d(in_channels, out_channels, (kernel_heights[2], embedding_length), stride, padding)
		self.dropout = nn.Dropout(keep_probab)
		self.label = nn.Linear(len(kernel_heights)*out_channels, output_size)
	
	def conv_block(self, input, conv_layer):
		conv_out = conv_layer(input)# conv_out.size() = (batch_size, out_channels, dim, 1)
		activation = F.relu(conv_out.squeeze(3))# activation.size() = (batch_size, out_channels, dim1)
		max_out = F.max_pool1d(activation, activation.size()[2]).squeeze(2)# maxpool_out.size() = (batch_size, out_channels)
		
		return max_out
	
	def forward(self, input_sentences, batch_size=None):
		
		"""
		The idea of the Convolutional Neural Netwok for Text Classification is very simple. We perform convolution operation on the embedding matrix 
		whose shape for each batch is (num_seq, embedding_length) with kernel of varying height but constant width which is same as the embedding_length.
		We will be using ReLU activation after the convolution operation and then for each kernel height, we will use max_pool operation on each tensor 
		and will filter all the maximum activation for every channel and then we will concatenate the resulting tensors. This output is then fully connected
		to the output layers consisting two units which basically gives us the logits for both positive and negative classes.
		
		Parameters
		----------
		input_sentences: input_sentences of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for pos & neg class.
		logits.size() = (batch_size, output_size)
		
		"""
		
		input = self.word_embeddings(input_sentences)
		# input.size() = (batch_size, num_seq, embedding_length)
		input = input.unsqueeze(1)
		# input.size() = (batch_size, 1, num_seq, embedding_length)
		max_out1 = self.conv_block(input, self.conv1)
		max_out2 = self.conv_block(input, self.conv2)
		max_out3 = self.conv_block(input, self.conv3)
		
		all_out = torch.cat((max_out1, max_out2, max_out3), 1)
		# all_out.size() = (batch_size, num_kernels*out_channels)
		fc_in = self.dropout(all_out)
		# fc_in.size()) = (batch_size, num_kernels*out_channels)
		logits = self.label(fc_in)
		
		return logits


In [4]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
    
def train_model(model, train_iter, epoch):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.cuda()
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    steps = 0
    model.train()
    for idx, batch in enumerate(train_iter):
        text = batch.text[0]
        target = batch.label
        target = torch.autograd.Variable(target).long()
        if torch.cuda.is_available():
            text = text.cuda()
            target = target.cuda()
        if (text.size()[0] != 32):# One of the batch returned by BucketIterator has length different than 32.
            continue
        optim.zero_grad()
        prediction = model(text)
        loss = loss_fn(prediction, target)
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        loss.backward()
        clip_gradient(model, 1e-1)
        optim.step()
        steps += 1
        
        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

def eval_model(model, val_iter):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            text = batch.text[0]
            if (text.size()[0] != 32):
                continue
            target = batch.label
            target = torch.autograd.Variable(target).long()
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
            prediction = model(text)
            loss = loss_fn(prediction, target)
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)

### **Train Model**

In [5]:
# parameters setting
learning_rate = 2e-5
batch_size = 32 # Batch size for training and evaluation
output_size = 2 # Number of classes for classification: 2
in_channels = 1 # Number of input channels for CNN: 1
out_channels = 10 # Number of output channels/feature maps for CNN
kernel_heights = [5, 7, 9] # Kernel heights for CNN: 5, 7, 9
stride = 1 # Stride for CNN
padding = 0 # Padding for CNN
keep_probab = 0.5 # Probability of retaining an activation node during dropout operation
embedding_length = 300

In [6]:
model = CNN(batch_size,
            output_size, 
            in_channels, 
            out_channels, 
            kernel_heights, 
            stride, 
            padding, 
            keep_probab, 
            vocab_size, 
            embedding_length, 
            weights = word_embeddings
            )
loss_fn = F.cross_entropy

for epoch in range(10):
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc = eval_model(model, valid_iter)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')

Epoch: 1, Idx: 100, Training Loss: 0.6914, Training Accuracy:  59.38%
Epoch: 1, Idx: 200, Training Loss: 0.6054, Training Accuracy:  68.75%
Epoch: 1, Idx: 300, Training Loss: 0.6070, Training Accuracy:  68.75%
Epoch: 1, Idx: 400, Training Loss: 0.4575, Training Accuracy:  75.00%
Epoch: 1, Idx: 500, Training Loss: 0.5003, Training Accuracy:  78.12%
Epoch: 01, Train Loss: 0.550, Train Acc: 70.88%, Val. Loss: 0.454949, Val. Acc: 78.66%
Epoch: 2, Idx: 100, Training Loss: 0.5232, Training Accuracy:  75.00%
Epoch: 2, Idx: 200, Training Loss: 0.5008, Training Accuracy:  78.12%
Epoch: 2, Idx: 300, Training Loss: 0.6430, Training Accuracy:  68.75%
Epoch: 2, Idx: 400, Training Loss: 0.5149, Training Accuracy:  71.88%
Epoch: 2, Idx: 500, Training Loss: 0.4644, Training Accuracy:  71.88%
Epoch: 02, Train Loss: 0.459, Train Acc: 78.52%, Val. Loss: 0.437626, Val. Acc: 79.52%
Epoch: 3, Idx: 100, Training Loss: 0.3243, Training Accuracy:  90.62%
Epoch: 3, Idx: 200, Training Loss: 0.4291, Training Accu

### **Test Model**

In [7]:
test_loss, test_acc = eval_model(model, test_iter)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')


def predict_sentiment(model, sentence):
    with torch.no_grad():
        test_sen = np.asarray(sentence)
        test_sen = torch.LongTensor(test_sen)
        test_tensor = Variable(test_sen)
        test_tensor = test_tensor.cuda()
        model.eval()
        output = model(test_tensor, 1)
        out = F.softmax(output, 1)
        if (torch.argmax(out[0]) == 1):
            print ("Sentiment: Positive")
        else:
            print ("Sentiment: Negative")
    
''' Let us now predict the sentiment on a single sentence just for the testing purpose. '''
test_sen1 = "This is one of the best creation of Nolan. I can say, it's his magnum opus. Loved the soundtrack and especially those creative dialogues."
test_sen2 = "Ohh, such a ridiculous movie. Not gonna recommend it to anyone. Complete waste of time and money."

print(test_sen1)
test_sen1 = TEXT.preprocess(test_sen1)
test_sen1 = [[TEXT.vocab.stoi[x] for x in test_sen1]]
predict_sentiment(model, test_sen1)

print(test_sen2)
test_sen2 = TEXT.preprocess(test_sen2)
test_sen2 = [[TEXT.vocab.stoi[x] for x in test_sen2]]
predict_sentiment(model, test_sen2)

Test Loss: 0.442, Test Acc: 80.47%
This is one of the best creation of Nolan. I can say, it's his magnum opus. Loved the soundtrack and especially those creative dialogues.
Sentiment: Positive
Ohh, such a ridiculous movie. Not gonna recommend it to anyone. Complete waste of time and money.
Sentiment: Negative
